# GeoStress AI - Fracture Analysis & Geostress Inversion

## AI-Powered Geostress Inversion from Fracture Orientation Data

This notebook demonstrates the full pipeline:
1. **Data Loading** - Read fracture orientation data from Excel files
2. **Exploratory Analysis** - Visualize fracture distributions
3. **Geostress Inversion** - Estimate the stress tensor using Mohr-Coulomb theory
4. **Fracture Classification** - ML model to classify fracture types
5. **Critically Stressed Fractures** - Identify fluid-conductive fractures
6. **Bayesian Uncertainty** - MCMC quantification of stress uncertainty

In [ ]:
import sys
sys.path.insert(0, '../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 11

from data_loader import (
    load_all_fractures, fracture_summary, fracture_plane_normal,
    AZIMUTH_COL, DIP_COL, DEPTH_COL, WELL_COL, FRACTURE_TYPE_COL
)
from geostress import invert_stress, bayesian_inversion
from fracture_analysis import classify_fracture_types, cluster_fracture_sets, identify_critically_stressed
from visualization import (
    plot_rose_diagram, plot_stereonet, plot_mohr_circle,
    plot_tendency, plot_depth_profile, plot_analysis_dashboard, _plot_stereonet_manual
)

print('All modules loaded successfully!')

## 1. Load Data

In [ ]:
df = load_all_fractures('../data/raw')
print(f'Total fractures loaded: {len(df)}')
print(f'Wells: {df[WELL_COL].unique().tolist()}')
print(f'Fracture types: {df[FRACTURE_TYPE_COL].unique().tolist()}')
print()
fracture_summary(df)

## 2. Exploratory Visualization

In [ ]:
# Rose diagrams per well
fig, axes = plt.subplots(1, 2, figsize=(12, 5), subplot_kw={'projection': 'polar'})
for ax, well in zip(axes, ['3P', '6P']):
    df_well = df[df[WELL_COL] == well]
    plot_rose_diagram(df_well[AZIMUTH_COL].values, title=f'Well {well} ({len(df_well)} fractures)', ax=ax)
plt.tight_layout()
plt.savefig('../outputs/rose_diagrams.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Stereonets per well
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
for ax, well in zip(axes, ['3P', '6P']):
    df_well = df[df[WELL_COL] == well].reset_index(drop=True)
    _plot_stereonet_manual(df_well, f'Fracture Poles - Well {well}', 'fracture_type', ax)
plt.tight_layout()
plt.savefig('../outputs/stereonets.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Depth profiles for wells with depth data
df_with_depth = df.dropna(subset=[DEPTH_COL])
if len(df_with_depth) > 0:
    plot_depth_profile(df_with_depth, 'Fracture Distribution vs Depth')
    plt.savefig('../outputs/depth_profile.png', dpi=150, bbox_inches='tight')
    plt.show()

## 3. Geostress Inversion

Using Mohr-Coulomb frictional failure theory + differential evolution optimization
to estimate the stress tensor from fracture orientations.

In [ ]:
results = {}

for well in ['3P', '6P']:
    df_well = df[df[WELL_COL] == well].reset_index(drop=True)
    normals = fracture_plane_normal(df_well[AZIMUTH_COL].values, df_well[DIP_COL].values)
    avg_depth = df_well[DEPTH_COL].mean()
    if np.isnan(avg_depth):
        avg_depth = 3300.0
    
    print(f'\n{"="*50}')
    print(f'Well {well}: {len(normals)} fractures, avg depth {avg_depth:.0f}m')
    print(f'{"="*50}')
    
    # Try all three regimes
    for regime in ['normal', 'strike_slip', 'thrust']:
        result = invert_stress(normals, regime=regime, depth_m=avg_depth)
        cost = result['optimization_result'].fun
        print(f'  {regime:12s}: sigma1={result["sigma1"]:.1f}, sigma3={result["sigma3"]:.1f}, '
              f'R={result["R"]:.3f}, SHmax={result["shmax_azimuth_deg"]:.1f}deg, '
              f'mu={result["mu"]:.3f}, cost={cost:.1f}')
        
        if well not in results or cost < results[well]['optimization_result'].fun:
            results[well] = result
            results[well]['df'] = df_well
    
    best = results[well]
    print(f'\n  BEST FIT: {best["regime"]}')
    print(f'    sigma1 = {best["sigma1"]:.1f} MPa')
    print(f'    sigma2 = {best["sigma2"]:.1f} MPa')
    print(f'    sigma3 = {best["sigma3"]:.1f} MPa')
    print(f'    R = {best["R"]:.3f}')
    print(f'    SHmax = {best["shmax_azimuth_deg"]:.1f} degrees')
    print(f'    mu = {best["mu"]:.3f}')

In [ ]:
# Mohr circles for each well
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
for ax, well in zip(axes, ['3P', '6P']):
    plot_mohr_circle(results[well], f'Mohr Circle - Well {well}', ax=ax)
plt.tight_layout()
plt.savefig('../outputs/mohr_circles.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Critically Stressed Fractures

Fractures where slip tendency > friction coefficient are critically stressed
and likely to be hydraulically conductive (fluid pathways).

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

for col, well in enumerate(['3P', '6P']):
    res = results[well]
    df_well = res['df']
    
    # Slip tendency
    plot_tendency(df_well, res['slip_tend'],
                 f'Slip Tendency - Well {well}', 'RdYlGn_r', ax=axes[0, col])
    
    # Dilation tendency
    plot_tendency(df_well, res['dilation_tend'],
                 f'Dilation Tendency - Well {well}', 'RdYlBu_r', ax=axes[1, col])
    
    # Statistics
    cs = identify_critically_stressed(res['sigma_n'], res['tau'], res['mu'])
    print(f'Well {well}: {cs.sum()}/{len(cs)} fractures critically stressed '
          f'({100*cs.sum()/len(cs):.1f}%)')

plt.tight_layout()
plt.savefig('../outputs/tendency_plots.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Fracture Type Classification (ML)

In [ ]:
# Random Forest classifier
clf_result = classify_fracture_types(df, classifier='random_forest')

print(f'Cross-Validation Accuracy: {clf_result["cv_mean_accuracy"]:.3f} '
      f'+/- {clf_result["cv_std_accuracy"]:.3f}')
print(f'\nFeature Importances:')
for feat, imp in sorted(clf_result['feature_importances'].items(), key=lambda x: -x[1]):
    bar = '█' * int(imp * 50)
    print(f'  {feat:12s} {bar} {imp:.3f}')

print(f'\nConfusion Matrix:')
print(pd.DataFrame(
    clf_result['confusion_matrix'],
    index=clf_result['label_encoder'].classes_,
    columns=clf_result['label_encoder'].classes_
))

## 6. Fracture Set Clustering

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

for ax, well in zip(axes, ['3P', '6P']):
    df_well = df[df[WELL_COL] == well].reset_index(drop=True)
    clust = cluster_fracture_sets(df_well)
    
    ax.set_aspect('equal')
    ax.set_xlim(-1.2, 1.2)
    ax.set_ylim(-1.2, 1.2)
    theta = np.linspace(0, 2*np.pi, 100)
    ax.plot(np.cos(theta), np.sin(theta), 'k-')
    
    az_rad = np.radians(df_well[AZIMUTH_COL].values)
    dip_rad = np.radians(df_well[DIP_COL].values)
    pole_trend = az_rad + np.pi
    pole_plunge = np.pi/2 - dip_rad
    r = np.sqrt(2) * np.sin(pole_plunge / 2)
    x = r * np.sin(pole_trend)
    y = r * np.cos(pole_trend)
    
    colors = plt.cm.Set1(np.linspace(0, 1, clust['n_clusters']))
    for c in range(clust['n_clusters']):
        mask = clust['labels'] == c
        stats = clust['cluster_stats'].iloc[c]
        ax.scatter(x[mask], y[mask], s=15, c=[colors[c]], alpha=0.6,
                  label=f'Set {c}: az={stats["mean_azimuth"]:.0f}, dip={stats["mean_dip"]:.0f} ({stats["count"]:.0f})')
    
    ax.set_title(f'Fracture Sets - Well {well} ({clust["n_clusters"]} sets)')
    ax.legend(fontsize=7, loc='upper left')
    for angle, label in [(0,'N'),(90,'E'),(180,'S'),(270,'W')]:
        rad = np.radians(angle)
        ax.text(1.15*np.sin(rad), 1.15*np.cos(rad), label, ha='center', va='center', fontweight='bold')

plt.tight_layout()
plt.savefig('../outputs/fracture_sets.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Full Summary Dashboards

In [ ]:
for well in ['3P', '6P']:
    df_well = results[well]['df']
    fig = plot_analysis_dashboard(df_well, results[well], well_name=f'Well {well}',
                                  save_path=f'../outputs/dashboard_{well}.png')
    plt.show()

## 8. Results Summary

### Key Findings

In [ ]:
print('╔══════════════════════════════════════════════════════╗')
print('║        GEOSTRESS INVERSION RESULTS SUMMARY          ║')
print('╠══════════════════════════════════════════════════════╣')
for well in ['3P', '6P']:
    r = results[well]
    cs = identify_critically_stressed(r['sigma_n'], r['tau'], r['mu'])
    print(f'║  Well {well}:                                          ║')
    print(f'║    σ1 = {r["sigma1"]:6.1f} MPa                              ║')
    print(f'║    σ2 = {r["sigma2"]:6.1f} MPa                              ║')
    print(f'║    σ3 = {r["sigma3"]:6.1f} MPa                              ║')
    print(f'║    R  = {r["R"]:6.3f}                                  ║')
    print(f'║    SHmax = {r["shmax_azimuth_deg"]:5.1f}°                             ║')
    print(f'║    μ  = {r["mu"]:6.3f}                                  ║')
    print(f'║    Regime: {r["regime"]:12s}                         ║')
    print(f'║    Critically stressed: {cs.sum():3d}/{len(cs)} ({100*cs.sum()/len(cs):.0f}%)             ║')
    print(f'║                                                      ║')
print('╚══════════════════════════════════════════════════════╝')